In [1]:
import os.path

import cv2
import pandas as pd
import numpy as np
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class KNN:
    def __init__(self):
        csvPath = '/kaggle/input/butterfly-image-classification/Training_set.csv'
        self.imageDataSet = pd.read_csv(csvPath);
        self.labels = []
        self.labels = []
        self.getLabel()
        self.X = []
        self.Y = []
        self.name = []
        self.getData()
    def getLabel(self):
        self.labels = list(self.imageDataSet['label'].unique())

    def getData(self):
        for index, imageData in self.imageDataSet.iterrows():
            #print(index)
            filename = '/kaggle/input/butterfly-image-classification/train/' + imageData['filename']
            self.name.append(imageData['filename'])
            image = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
            if image is not None:
                image = image / 255.0
                image = image.flatten()
                label = imageData['label']
                self.X.append(image)
                self.Y.append(label)

                image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
                self.X.append(image.flatten())
                self.Y.append(label)

                image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
                self.X.append(image.flatten())
                self.Y.append(label)

                image = cv2.rotate(image, cv2.ROTATE_180)
                self.X.append(image.flatten())
                self.Y.append(label)
            
            del image
        
    def train(self):
        self.le = LabelEncoder()
        print("hi")
        self.le.fit(self.Y)
        encoded_Y = self.le.transform(self.Y)
        del self.Y
        self.num_classes = len(self.le.classes_)
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(
            self.X, encoded_Y, test_size=0.1, random_state=42
        )
        del self.X
        del encoded_Y


        self.clf = KNeighborsClassifier(n_neighbors=1,metric='euclidean')
        self.clf.fit(self.X_train, self.Y_train)

    def evaluate(self):
        Y_pred = self.clf.predict(self.X_test)
        accuracy = accuracy_score(self.Y_test, Y_pred)
        print("Test accuracy:", accuracy)
        #for i in range(0,len(self.Y_test)):
        #    print(self.labels[Y_pred[i]]+" : "+self.labels[self.Y_test[i]])

    def evaluateImage(self,image):
        image = np.array(image)
        if len(image.shape) == 3 and image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        image = image / 255.0
        image = image.flatten()
        image=image.reshape(1, -1)
        Y_pred = self.clf.predict(image)
        return self.labels[Y_pred[0]]

    def saveNewTrainImage(self,image,fileName,label):
        try:
            image.save(os.path.join('.' + self.location + '/dataset/train/' + fileName), fileName)


        except Exception as e:
            return e

    def saveNewTrainImageData(self,fileName,label):
        try :
            with open('.' + self.location + '/dataset/Training_set.csv', mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([fileName, label])
        except Exception as e:
            return e

node = KNN()
node.train()
node.evaluate()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


hi
Test accuracy: 0.9123076923076923
